In [ ]:
# Install  non-standard library to analyse Kepler data
!pip install lightkurve

<h1>Finding planets with the Kepler space mission data</h1>

The data from the Kepler space mission is publicly available, and there is an easy-to-use Python package to do some basic analysis, which we will use in this notebook. For more information, and a lot of examples, see [here](https://docs.lightkurve.org/index.html). Below, the very basics are shown of how to get from raw Kepler data to a planetary candidate. **There are questions in boldface.** There are boxes below the questions where you can type in your answers (double-click to enter text). Once you are done, download the notebook (File->Download->Download .ipynb) and hand it in on Brightspace.

<h2>Raw light curve</h2>

First, we select the raw data of a particular Kepler system, Kepler-10 by default, but you can easily change the name to look at a different system. We download the 'pixelfile', and can then look at the flux of each pixel as a function of time. Can you see some brightness variations in any of the pixels?

In [ ]:
import numpy as np
import lightkurve as lk
import matplotlib.pyplot as plt
%matplotlib inline

# Name of the object we are considering. 
name = "Kepler-10"

# Search for Kepler lightcurves at long cadence
search_result = lk.search_lightcurve(name, author='Kepler', cadence='long')
print(search_result)

# Download the pixel file and show animation. 
tpf = lk.search_targetpixelfile(name, quarter=3, cadence='long').download()
tpf.animate()

From these pixel values, we can create a light curve by adding up the values of all pictures in a single frame, so that we get a single number for every time frame: the brightness of the star. We can plot this flux as a function of time to detect any transiting planets. However, first of all, we need to detrend the data. 

Kepler observations are divided into quarters. In order to search for planets around other stars, the Kepler space telescope performed near-continuous monitoring of a single field of view, from an Earth-trailing orbit. However, this posed a challenge. If the space telescope is trailing Earth and maintaining steady pointing, its solar panels would slowly receive less and less sunlight. In order to make sure the solar panels remained oriented towards the Sun, Kepler performed quarterly rolls, one every 93 days. After each roll, Kepler retained its fine-pointing at the same field of view. Because the camera rotated by 90 degrees, all of the target stars fell on different parts of the charge-coupled device (CCD) camera. This had an effect on the amount of flux recorded for the same star, because different CCD pixels have different sensitivities. Therefore, a different quarter may give a different brightness for the same star!

In the figure below, each quarter of data looks strikingly different, with global patterns repeating every four quarters as Kepler has made a full rotation. The change in flux within each quarter is in part driven by changes in the telescope focus, which are caused by changes in the temperature of Kepler’s components as the spacecraft orbits the Sun. The changes are also caused by an effect called differential velocity aberration (DVA), which causes stars to drift over the course of a quarter, depending on their distance from the center of Kepler’s field of view.

While the figure above looks messy, all the systematic effects mentioned above are well understood, and can be detrended. 

**1(a) Look at the list above as to which quarters are available for this system. Execute the code in the block below. In the figure you make below, can you spot any patterns for quarters that are 4 apart, i.e. Q1, Q5, Q9, Q13, or Q3, Q7, Q11, Q15? What could be the reason for any similarities, and a possible reason for differences? (1 point)**

Answer: 

In [ ]:
lc_collection = search_result.download_all()
ax = lc_collection[0].plot(column='sap_flux', label=None)
for lc in lc_collection[1:]:
    lc.plot(ax=ax, column='sap_flux', label=None)

<h2>Flat Lightcurve</h2>

Now, we can subtract the trend from the lightcurve, to obtain what is sometimes called a 'flat lightcurve'. The most obvious transits (i.e. Hot Jupiters) can be easily spotted at this point if they are present. Note that there are still systematic offsets between the quarters!

In [ ]:
lc_collection.plot();

In order to get rid of the offset, we can simply 'stitch' all lightcurves together. In addition, we can do some more flattening to correct for any leftover trends, and remove the most obvious outliers (sharp spikes, perhaps caused by cosmic rays hitting the CCD).

**1(b) Execute the code below, both with and without removing the outliers. At what time can you see a possible cosmic ray hitting the camera? What could be a simple algorithm to detect such an event? (1 point)**

Answer: 

In [ ]:
lc = lc_collection.stitch().flatten(window_length=901).remove_outliers()
ax = lc.plot();
# Optionally, set the limits on the x-axis
#ax.set_xlim([-2,2])

<h2>Determining the period</h2>

The next step is to determine whether there is a periodic dimming present in the flat lightcurve. You can think of this as doing a [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) and look for the period where there is the highest power. In cases where the signal is very far from sinusoidal, other algorithms perform better than the Fourier transform. Here, we use a special algorithm for transits, called the [Box Least Squares](https://docs.astropy.org/en/stable/timeseries/bls.html) periodogram. Its output is very similar to a Fourier transform: power as a function of period (or frequency). 

We search in a range of periods, defined by the array ```period```. The range may have to be adjusted for different systems: often it is useful to start with a wide range, and then narrow in on the real period. 

**1(c) Execute the code below. What period does it find, and how does this compare to the published value for Kepler 10b (see [here](https://en.wikipedia.org/wiki/Kepler-10))? Now change the period search range to (0.5,1) days. What period does it find now, and how does it relate to the originally found period? Can you think of a reason why the algorithm found this original period? (2 points)**

Answer: 

In [ ]:
# Create array of periods to search (between 0.5 and 1 day)
period = np.linspace(1, 10, 10000)
# Create a BLSPeriodogram
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500);
bls.plot();

planet_b_period = bls.period_at_max_power
planet_b_t0 = bls.transit_time_at_max_power
planet_b_dur = bls.duration_at_max_power

# Check the value for period
planet_b_period

<h2>Folding the lightcurve</h2>

We can use the period $P$ where the BLS periodogram attains maximum power to create a 'folded' lightcurve. After every $P$, we reset the time to the initial time $t_0$, so that time runs from $t_0$ to $t_0+P$ only. This time interval is then renamed 'phase'. If the total observation time is 10 $P$ (say), this means that we get 10 lightcurves on top of each other. If a strong periodic signal is present, these lightcurves will strongly overlap, so that a transit is easy to see. 

**1(d) The red curve is the transit model. Look up the parameters of the star [Kepler-10](https://en.wikipedia.org/wiki/Kepler-10). Calculate the radius of the planet from the transit figure. How does it compare to the published radius? Can you think of any reasons for the potential difference? (1 point)**

Answer: 

In [ ]:
ax = lc.fold(period=planet_b_period, epoch_time=planet_b_t0).scatter()

# Create a BLS model using the BLS parameters
planet_b_model = bls.get_transit_model(period=planet_b_period,
                                       transit_time=planet_b_t0,
                                       duration=planet_b_dur)

planet_b_model.fold(planet_b_period, planet_b_t0).plot(ax=ax, c='r', lw=2);

Of course, there may be more than on transiting planet in the system. To look for additional planets, we can mask the first planet, and do a new period search. Note that we are using a different period range here, based on our knowledge of the Kepler-10 system.

In [ ]:
# Create a cadence mask using the BLS parameters
planet_b_mask = bls.get_transit_mask(period=planet_b_period,
                                     transit_time=planet_b_t0,
                                     duration=planet_b_dur)
masked_lc = lc[~planet_b_mask]
#ax = masked_lc.scatter();
#lc[planet_b_mask].scatter(ax=ax, c='r', label='Masked');

period = np.linspace(1, 100, 10000)
bls = masked_lc.to_periodogram('bls', period=period, frequency_factor=500)
bls.plot();

planet_c_period = bls.period_at_max_power
planet_c_t0 = bls.transit_time_at_max_power
planet_c_dur = bls.duration_at_max_power

# Check the value for period
planet_c_period

As before, we can plot the folded lightcurve. In this case, we do not plot the transit model but the binned flux (i.e. the average over some window). 

**1(e) Compare the period found to that of Kepler-10c (see again [Wikipedia](https://en.wikipedia.org/wiki/Kepler-10)). Execute the code below. As above, work out the planet radius, and compare with the published value. Comment on any differences. (1 point)**

Answer: 

In [ ]:
ax = masked_lc.fold(planet_c_period, planet_c_t0).scatter()
masked_lc.fold(planet_c_period, planet_c_t0).bin(.1).plot(ax=ax, c='r', lw=2,
                                                          label='Binned Flux');

**2 Go back to the top, and look at the potential moon-harboring planet around the star Kepler-1708.**

**(a) Go through the same steps as above, and determine the period and radius of the planet Kepler-1708b. There is no need to look for a second planet. Compare your findings with the published values on [Wikipedia](https://en.wikipedia.org/wiki/Kepler-1708b). Hint: you may want to do a basic stitching, without removing outliers or further flattening! (1 point)**

Answer:

**(b) How many transit events can you find in the unfolded lightcurve? Write down the times at which they occur. (1 point)**

Answer:

**(c) Zoom in on these transit events, and compare them to those shown in the lecture slides on Kepler-1708b-i. Leave your favourite transit of Kepler 1708b plotted below question 1b. In two sentences, explain how convinced you are that this planet does in fact have an exomoon. (2 points)**

Answer: